<a href="https://colab.research.google.com/github/gitwail/OpenPortfolio/blob/main/Mingpt_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-07-25 14:13:14--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2024-07-25 14:13:14 (18.0 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [2]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


# Preprocessing dataset

## Encoding and Decoding

In [4]:
# finding the vocabulary
vocab=sorted(set(list(text)))
print("vocab size is:",len(vocab))
print(vocab)

vocab size is: 65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [5]:
# Mapping dict converting string and integers
stoi= {c:i for i,c in enumerate(vocab)}
itos={i:c for i,c in enumerate(vocab)}

In [6]:
# Encoding/decoding functions that convert a sentence and tokens (integers here !)
encode= lambda sentence:[ stoi[s] for s in sentence]
decode= lambda tokens:[ itos[t] for t in tokens]

In [7]:
sentence="hello my name is wail"
print(sentence)
tokens=encode(sentence)
print(tokens)
print(decode(tokens))

hello my name is wail
[46, 43, 50, 50, 53, 1, 51, 63, 1, 52, 39, 51, 43, 1, 47, 57, 1, 61, 39, 47, 50]
['h', 'e', 'l', 'l', 'o', ' ', 'm', 'y', ' ', 'n', 'a', 'm', 'e', ' ', 'i', 's', ' ', 'w', 'a', 'i', 'l']


## Splitting the dataset (90% train, 10% validation)

In [8]:
dataset_len=len(text)
split_indice=int(dataset_len*0.9)
train=text[:split_indice]
validation=text[split_indice:]


# Sampling batch

In [9]:
import torch
import torch.nn as nn
from torch.nn import functional as F


In [10]:
# encode and torchify datasets
train=torch.tensor(encode(train))
validation=torch.tensor(encode(validation))

In [11]:
# batch size
batch_size=32
# context window
context_window=8

def get_batch(split='train',batch_size=32,context_window=8):
    # generate a small batch of data of inputs x and targets y
    data = train if split == 'train' else validation
    ix = torch.randint(len(data) - context_window, (batch_size,))
    # print("random ints generated: ",ix)
    x = torch.stack([data[i:i+context_window] for i in ix])
    y = torch.stack([data[i+1:i+context_window+1] for i in ix])
    return x, y

x,y=get_batch(split='train',batch_size=batch_size,context_window=context_window)
print(f"input:{x}")
print(f"target:{y}")


input:tensor([[52, 48, 59, 57, 58,  1, 59, 52],
        [58,  1, 21,  1, 41, 53, 52, 42],
        [57, 47, 56, 43,  1, 58, 53,  1],
        [32, 46, 43, 63,  1, 51, 59, 57],
        [14, 59, 58,  1, 41, 53, 51, 43],
        [41, 58, 57,  1, 61, 47, 58, 46],
        [26, 17, 26, 21, 33, 31, 10,  0],
        [ 1, 21,  1, 51, 59, 57, 58,  1],
        [43, 56,  6,  1, 63, 53, 59,  1],
        [58, 46,  1, 57, 46, 43,  1, 52],
        [35, 46, 47, 41, 46,  1, 21,  1],
        [56,  1, 53, 44,  1, 46, 47, 57],
        [56, 43,  1, 41, 53, 51, 43,  1],
        [44,  1, 46, 47, 57,  1, 50, 39],
        [ 1, 43, 58, 43, 56, 52, 39, 50],
        [ 1, 51, 63,  1, 42, 43, 39, 56],
        [63, 53, 59,  8,  0,  0, 14, 53],
        [58, 59, 52, 43,  1, 58, 47, 50],
        [46, 47, 57,  1, 41, 53, 59, 52],
        [47, 40, 59, 52, 43,  8,  0,  0],
        [42,  1, 41, 53, 51, 54, 39, 57],
        [50, 50, 57, 10,  0, 13, 52, 42],
        [ 1, 39, 44, 58, 43, 56,  1, 56],
        [ 1, 41, 53, 59, 50,

# GPT model

## BigramModel

In [100]:
################# hyperparams and params #########################

# vocal size is 65
vocab_size=len(vocab)
# batch size
batch_size=32
# context window
context_window=8
# n_embding
n_embd=32
# multihead number of heads
n_heads=4


In [101]:

class MultiHeadAttention(nn.Module):
  def __init__(self,head_size,n_head):
    super(MultiHeadAttention,self).__init__()
    self.heads=nn.ModuleList([Head(head_size//n_head) for i in range(n_head)])

  def forward(self,x):
    list_heads=[h(x) for h in self.heads ] # (B,T,H)

    return torch.concat(list_heads,dim=-1)



class Head(nn.Module):
  def __init__(self,head_size):
    super(Head,self).__init__()
    self.Q=nn.Linear(n_embd,head_size)
    self.K=nn.Linear(n_embd,head_size)
    self.V=nn.Linear(n_embd,head_size)
    self.register_buffer('tril', torch.tril(torch.ones(context_window,context_window)))


  def forward(self,x):
    # x (B,T,C)
    B,T,C=x.shape
    q=self.Q(x) # (B,T,H)
    k=self.K(x) # (B,T,H)
    v=self.V(x) # (B,T,H)
    wei=q@k.transpose(-2,-1) * (k.shape[-1]**-0.5) # (B,T,H)@(B,H,T) --> (B,T,T)
    # masking the weight matrix
    wei=wei.masked_fill(self.tril[:T,:T]==0,float('-inf')) # (B,T,T)
    wei=F.softmax(wei,dim=-1) # (B,T,T)
    out=wei@v # (B,T,T) @ (B,T,H) --> (B,T,H)
    return out




class BigramModel(nn.Module):
  def __init__(self):
    super(BigramModel,self).__init__()
    self.embeding=nn.Embedding(vocab_size,n_embd)
    self.pos_enc=nn.Embedding(context_window,n_embd)
    self.mha=MultiHeadAttention(head_size=n_embd,n_head=n_heads)
    self.lin=nn.Linear(n_embd,vocab_size)



  def forward(self,x,target=None):
    # x  (B,T) and target (B,T)
    x=self.embeding(x) # (B,T,C)
    B,T,C=x.shape
    # positional encoding
    x=x+self.pos_enc(torch.arange(T))# (B,T,C)
    # logits
    logits=self.mha(x)
    logits=self.lin(logits)


    if target!=None:
      B,T,C= logits.shape
      logits=logits.view(B*T,C)
      target=target.view(B*T)
      loss=F.cross_entropy(logits,target)
    else:
      loss=None

    return logits,loss

  def generate(self,ix,max_new_tokens=100):

    for _ in range(max_new_tokens):
       # (B,T)
      logits,_=self(ix[:,-context_window:]) # (B,T,V)
      proba=F.softmax(logits[:,-1,:],dim=-1) #(B,V)
      ix_new=torch.multinomial(proba,1,replacement=True)
      ix=torch.concat((ix,ix_new),dim=1)

    return ix


model=BigramModel()
logits,loss=model(x,y)
# x[0,0]

In [90]:
mha=MultiHeadAttention(head_size=32,n_head=4)
mha(nn.Embedding(vocab_size,n_embd)(x)).shape

torch.Size([32, 8, 32])

## Training loop

In [22]:
print("".join(decode(model.generate(torch.zeros((1,1),dtype=torch.long))[0].tolist())))


jF-bxLIGsBWvCOXkR
?fsqeC,rg3:;FPxW,RGRas:tcBIRAwk&KBsus;CJJ3N3$AAwOrln;GOS,ejENISgrL?r dDiwEoT:WL'$;


In [102]:
model=BigramModel()


In [103]:
from torch.optim import AdamW

#seed
torch.manual_seed(1337)
# number of iteration
n_iteration=10000
# learning rate
lr=1e-3
# optimizer
optimizer=AdamW(model.parameters(),lr=1e-3)
# model instantiation
# model=BigramModel()

for i in range(n_iteration):
  # get batch
  x,y=get_batch(split='train')
  # flush the gradients
  optimizer.zero_grad()
  # loss
  logits,loss=model(x,y)
  # backward propagation
  loss.backward()
  # optimizer step
  optimizer.step()

  if i%1000==0: print(f"step: {i} training loss: {loss.item()}")

step: 0 training loss: 4.194070339202881
step: 1000 training loss: 2.5240063667297363
step: 2000 training loss: 2.3990373611450195
step: 3000 training loss: 2.279484748840332
step: 4000 training loss: 2.2065703868865967
step: 5000 training loss: 2.3841466903686523
step: 6000 training loss: 2.150954246520996
step: 7000 training loss: 2.1387007236480713
step: 8000 training loss: 2.1599550247192383
step: 9000 training loss: 2.124406576156616


In [104]:
print("".join(decode(model.generate(torch.zeros((1,1),dtype=torch.long),max_new_tokens=300)[0].tolist())))


for verni; worid's quand my nemped dis to sices
Of I
Nongord well and me ase men?
Whe pha wamak, my's you Weemsbe shovind the hey can well mue ord int
And and the sivel
UTher the sude mor dean:
Crwis jurell sprame I I Isans onur gonepelene,
WARINCESTANRY:
Dies tioped a I.'ls ward of ut
Orie Meaune i
